In [1]:
import os

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Input, Dropout, Conv2D, MaxPooling2D, Flatten, Reshape, GaussianNoise, Conv2DTranspose, LeakyReLU
from tensorflow.keras.constraints import MinMaxNorm
from tensorflow.keras import Sequential

from sklearn.preprocessing import StandardScaler

import sys
sys.path.append("../wann/methods")
from WANN import WANN

from adapt.instance_based import KLIEP, KMM, TrAdaBoostR2
from adapt.feature_based import MDD, DANN, ADDA, DeepCORAL

from warnings import filterwarnings
filterwarnings('ignore')
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [2]:
def concat_files(domains):   
    for domain in domains:
        path = "../dataset/citycam/%s"%domain
        for r, d, f in os.walk(path):
            for direct in d:
                if "checkpoints" not in direct:
                    x_path = path + "/" + direct + "/" + "X.npy"
                    y_path = path + "/" + direct + "/" + "y.npy"

                    Xi = np.load(x_path)
                    yi = np.load(y_path)
                    
                    if len(yi) != len(Xi):
                        print(len(yi), len(Xi))

                    try:
                        X = np.concatenate((X, Xi))
                        y = np.concatenate((y, yi))
                    except:
                        X = np.copy(Xi)
                        y = np.copy(yi)
    return X, y

In [3]:
def get_base_model(shape=2048, activation=None, C=1., name="BaseModel"):
    inputs = Input(shape=(shape,))
    modeled = Dense(100, activation='relu',
                         kernel_constraint=MinMaxNorm(0, C),
                         bias_constraint=MinMaxNorm(0, C))(inputs)
    modeled = Dense(100, activation='relu',
                    kernel_constraint=MinMaxNorm(0, C),
                    bias_constraint=MinMaxNorm(0, C))(modeled)
    modeled = Dense(1, activation=activation,
                    kernel_constraint=MinMaxNorm(0, C),
                    bias_constraint=MinMaxNorm(0, C))(modeled)
    model = Model(inputs, modeled, name=name)
    model.compile(optimizer=Adam(0.001), loss='mean_squared_error')
    return model


def get_encoder(shape=2048, C=1, name="encoder"):
    inputs = Input(shape=(shape,))
    modeled = Dense(100, activation='relu',
                         kernel_constraint=MinMaxNorm(0, C),
                         bias_constraint=MinMaxNorm(0, C))(inputs)
    model = Model(inputs, modeled)
    model.compile(optimizer="adam", loss='mean_squared_error')
    return model


def get_task(shape=100, C=1, activation=None, name="task"):
    inputs = Input(shape=(shape,))
    modeled = Dense(100, activation='relu',
                    kernel_constraint=MinMaxNorm(0, C),
                    bias_constraint=MinMaxNorm(0, C))(inputs)
    modeled = Dense(1, activation=activation,
                         kernel_constraint=MinMaxNorm(0, C),
                         bias_constraint=MinMaxNorm(0, C))(modeled)
    model = Model(inputs, modeled)
    model.compile(optimizer="adam", loss='mean_squared_error')
    return model

In [4]:
source = ["253", "511", "572"]
target = ["495"]

Xs, ys = concat_files(source)
Xt, yt = concat_files(target)

Xs = Xs.astype(np.float64)
Xt = Xt.astype(np.float64)
ys = ys.astype(np.float64)
yt = yt.astype(np.float64)

X = np.concatenate((Xs, Xt))
y = np.concatenate((ys, yt))

src_index = np.array(range(len(ys)))
tgt_index = np.array(range(len(ys), len(ys) + len(yt)))

std_sc = StandardScaler()
std_sc.fit(X[src_index])

X = std_sc.transform(X)

mu = ys.mean()
std = ys.std()

y = (y - mu) / std

In [6]:
np.random.seed(0)
seeds = np.random.choice(2**16, 10)

epochs = 100
batch_size = 1000

for n_target_labeled in [20, 50, 100, 200]:

    scores = {}

    for i in range(10):

        np.random.seed(seeds[i])
        tf.random.set_seed(seeds[i])

        tgt_index_labeled = np.random.choice(tgt_index, n_target_labeled, replace=False)
        train_index = np.concatenate((src_index, tgt_index_labeled))

        for method in ["WANN", "TgtOnly", "NoReweight", "KLIEP", "KMM", "DANN", "ADDA", "DeepCORAL", "MDD", "TrAdaBoostR2"]: # "WANN", "TgtOnly", "NoReweight", "KLIEP", "KMM", "DANN", "ADDA", "DeepCORAL", "MDD", "TrAdaBoostR2"
            print(method)
            if not method in scores:
                scores[method] = []

            if method in ["TgtOnly", "NoReweight"]:            
                model = get_base_model(shape=X.shape[1])

                if method == "TgtOnly":
                    model.fit(X[tgt_index_labeled], y[tgt_index_labeled], epochs=epochs, batch_size=batch_size, verbose=0)
                if method == "NoReweight":
                    model.fit(X[train_index], y[train_index], epochs=epochs, batch_size=batch_size, verbose=0)

            elif method in ["WANN"]:
                model = WANN(get_base_model=get_base_model, C=1., optimizer=Adam(0.001))   
                model.fit(X, y, [train_index, tgt_index_labeled], epochs=epochs, verbose=0, batch_size=batch_size)
            
            elif method == "TrAdaBoostR2":
                model = TrAdaBoostR2(get_base_model(), verbose=2, random_state=seeds[i])
                model.fit(X[train_index], y[train_index], X[tgt_index_labeled], y[tgt_index_labeled],
                          epochs=epochs, batch_size=batch_size, verbose=0)

            else:            
                if method == "DANN":
                    model = DANN(encoder=get_encoder(), task=get_task(), random_state=seeds[i],
                                 discriminator=get_task(activation="sigmoid"),
                                 optimizer=Adam(0.001), lambda_=0.1, loss="mse")
                if method == "DeepCORAL":
                    model = DeepCORAL(encoder=get_encoder(), task=get_task(), lambda_=10.,
                                      optimizer=Adam(0.001), loss="mse",
                                      random_state=seeds[i])
                if method == "MDD":
                    model = MDD(encoder=get_encoder(), task=get_task(), random_state=seeds[i],
                                optimizer=Adam(0.001), lambda_=0.0001, loss="mse")
                if method == "ADDA":
                    encoder = get_encoder()
                    task=get_task()
                    discriminator=get_task(activation="sigmoid")
                    dann = DANN(encoder, task, discriminator, loss="mse", copy=False,
                                lambda_=0., random_state=seeds[i])
                    dann.fit(X[train_index], y[train_index], X[tgt_index], epochs=epochs, batch_size=batch_size, verbose=0)
                    model = ADDA(encoder=encoder, task=task,
                                 discriminator=discriminator, random_state=seeds[i],
                                 is_pretrained=True,
                                 optimizer=Adam(0.0001), loss="mse")
                if method == "KLIEP":
                    model = KLIEP(get_base_model(), sigmas=0.001, random_state=seeds[i])
                if method == "KMM":
                    model = KMM(get_base_model(), kernel_params=dict(gamma=0.001), verbose=0, random_state=seeds[i])

                model.fit(X[train_index], y[train_index], X[tgt_index], epochs=epochs, batch_size=batch_size, verbose=0)

            err_s = np.mean(np.abs(model.predict(X).ravel()[src_index] - y[src_index]) * std)
            err_t = np.mean(np.abs(model.predict(X).ravel()[tgt_index] - y[tgt_index]) * std)

            scores[method].append(err_t)

            print("source %.3f"%err_s)
            print("target %.3f"%err_t)
    
    print(pd.DataFrame(scores).describe())
    pd.DataFrame(scores).to_csv("../dataset/results/citycam_%i.csv"%n_target_labeled)